In [34]:
import torch    
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.datasets import fetch_covtype
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score

In [35]:
# Učitavanje podataka
covertype = fetch_covtype()
X = covertype.data
y = covertype.target

# Normalizacija
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Enkodovanje ciljne varijable da počinje od nule
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Podjela skupa podataka na trening, validaciju i test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Konvertovanje podataka u "PyTorch" tenzore
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).long()
X_val_tensor = torch.from_numpy(X_val).float()
y_val_tensor = torch.from_numpy(y_val).long()
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).long()

In [36]:
# Pravljenje "DataLoader"-a za trening i validacione skupove
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = torch.utils.data.TensorDataset(X_val_tensor, y_val_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

In [37]:
# Definicija arhitekture neuronske mreže
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [38]:
# Hiperparametri
hidden_size = 64
num_epochs = 10
learning_rate = 0.01
batch_size = 32

# Pravljenje modela
model = NeuralNetwork(X_train.shape[1], hidden_size, len(label_encoder.classes_))

# Definicija funkcije gubitka i optimizera
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# "Early stopping" parametri
patience = 2
max_val_loss = float('inf')
consecutive_no_improvement = 0

# Trening petlja
for epoch in range(num_epochs):  
    # Trening
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validacija
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()

    val_loss /= len(val_loader)
    
    print(f'Epoha {epoch+1}, Gubici: {loss}, Validacioni gubici: {val_loss}')

    # Provjera zа "Early stopping"
    if val_loss < max_val_loss:
        max_val_loss = val_loss
        consecutive_no_improvement = 0
    else:
        consecutive_no_improvement += 1
        if consecutive_no_improvement >= patience:
            break

Epoha 1, Gubici: 0.45397090911865234, Validacioni gubici: 0.7146528561978364
Epoha 2, Gubici: 0.35417652130126953, Validacioni gubici: 0.5112242819222187
Epoha 3, Gubici: 0.17353059351444244, Validacioni gubici: 0.5159052336987182
Epoha 4, Gubici: 0.47375258803367615, Validacioni gubici: 0.5085381660597618
Epoha 5, Gubici: 0.45767074823379517, Validacioni gubici: 0.49742467606059454
Epoha 6, Gubici: 0.5737228989601135, Validacioni gubici: 0.529943787597289
Epoha 7, Gubici: 0.504175066947937, Validacioni gubici: 0.4763938982170447
Epoha 8, Gubici: 0.5334192514419556, Validacioni gubici: 0.48893965286678964
Epoha 9, Gubici: 0.8234027624130249, Validacioni gubici: 0.4923232944101526


In [39]:
# Evaluacija modela
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = accuracy_score(y_test_tensor, predicted)
    recall = recall_score(y_test_tensor, predicted, average='macro')

print("Preciznost:", accuracy)
print("Odziv:", recall)


Preciznost: 0.7979226009655517
Odziv: 0.6194384672002623


In [41]:
# Hiperparametri
hidden_sizes = [64, 128]
num_epochs = 10
learning_rates = [0.001, 0.01]
batch_sizes = [32, 64]

best_accuracy = 0.0
best_model = None

# Pretraga mreže
for hidden_size in hidden_sizes:
    for learning_rate in learning_rates:
        for batch_size in batch_sizes:
            
            model = NeuralNetwork(X_train.shape[1], hidden_size, len(label_encoder.classes_))

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)

            patience = 2
            max_val_loss = float('inf')
            consecutive_no_improvement = 0

            for epoch in range(num_epochs):
                model.train()
                for inputs, labels in train_loader:
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                model.eval()
                val_loss = 0.0
                with torch.no_grad():
                    for inputs, labels in val_loader:
                        outputs = model(inputs)
                        val_loss += criterion(outputs, labels).item()

                val_loss /= len(val_loader)

                print(f'Epoha {epoch+1}, Gubici: {loss}, Validacioni gubici: {val_loss}')
                
                if val_loss < max_val_loss:
                    max_val_loss = val_loss
                    consecutive_no_improvement = 0
                else:
                    consecutive_no_improvement += 1
                    if consecutive_no_improvement >= patience:
                        break

            model.eval()
            with torch.no_grad():
                outputs = model(X_test_tensor)
                _, predicted = torch.max(outputs.data, 1)
                accuracy = accuracy_score(y_test_tensor, predicted)
                recall = recall_score(y_test_tensor, predicted, average='macro')

            print(f"Skrivena velicina: {hidden_size}, Stopa ucenja: {learning_rate}, Velicina serije: {batch_size}")
            print("Preciznost:", accuracy)
            print("Odziv:", recall)

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model = model

#Korišćenje najboljeg modela
best_model.eval()

Epoha 1, Gubici: 0.9407287240028381, Validacioni gubici: 0.5396290150075556
Epoha 2, Gubici: 0.35664528608322144, Validacioni gubici: 0.5041989380389924
Epoha 3, Gubici: 0.28249964118003845, Validacioni gubici: 0.4794100130262739
Epoha 4, Gubici: 0.6893730759620667, Validacioni gubici: 0.47430595438299267
Epoha 5, Gubici: 0.46401455998420715, Validacioni gubici: 0.4656092771069395
Epoha 6, Gubici: 0.2242753952741623, Validacioni gubici: 0.4564485052204386
Epoha 7, Gubici: 0.18437744677066803, Validacioni gubici: 0.44829196840811825
Epoha 8, Gubici: 0.2466065138578415, Validacioni gubici: 0.43576832758922945
Epoha 9, Gubici: 0.47224515676498413, Validacioni gubici: 0.43532277825971105
Epoha 10, Gubici: 0.1797928512096405, Validacioni gubici: 0.4333840807937009
Skrivena velicina: 64, Stopa ucenja: 0.001, Velicina serije: 32
Preciznost: 0.8193678304346703
Odziv: 0.6929257118403227
Epoha 1, Gubici: 0.4641112685203552, Validacioni gubici: 0.5387693121875473
Epoha 2, Gubici: 1.30760455131530

NeuralNetwork(
  (fc1): Linear(in_features=54, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=7, bias=True)
  (relu): ReLU()
)